## Script 
This script reads a input file name "input.dat" and returns top 5 matching data from the knowledge graph. 

## The knowledge graph 
It is hosted in the graph database neo4j. With py2neo, a package which helps to connect neo4j with python, we connect to the database. We run the query to return the most matching words from the list of query words named "input.dat". Before running this script, you must have loaded the knowledge graph(db.csv) in the neo4j graph and should be running. The instructions for loading the knowledge graph is mentioned in the README.md file in github. 

## Format for input.dat
 The file must contain query words seperated by commas. And the query words must be contained in the graph. For eg:
 fawn,pet

In [75]:
import time
from py2neo import Graph, Node
graph = Graph(password = "rosebay")

In [76]:
words = [word.split(',') for word in open("input.dat","r").readlines()]

In [81]:
# Todo : Return mismatch message if no word matches in the knowledge graph
result = open("results.csv","w")
for each in words[0]:
    print("--------------------------------------------")
    print(each)
    result.write("\n----------------\n")
    result.write(each)
    result.write("\n----------------\n")
    print("--------------------------------------------")
    query1 = '''
MATCH (n:Word)-[r]->(n2:Word) where n.name= '%s' RETURN n2,r.weight order by r.weight desc limit 5
    '''%each
    data = graph.run(query1).data()
    for each in data:
        print(each['n2']['name'],each['r.weight'])
        result.write("\n")
        result.write("%s,%s" %(each['n2']['name'],each['r.weight']))
        result.write("\n")

--------------------------------------------
fawn
--------------------------------------------
deer 6.500
calf 6.200
father 4.440
brown 4.340
color 3.920
--------------------------------------------
pet
--------------------------------------------
rabbit 9.830
cat 9.530
dog 9.230
animal 8.650
parrot 8.300


# Format of the result:
The result is displayed above and also written to a csv file named "results.csv". The format of the result is as shown above.